<a href="https://colab.research.google.com/github/kategavrishina/info-search/blob/master/HW1/sem1_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Семинар 1 Индекс

## Intro

### работа с файлами и папками

In [ ]:
import os

curr_dir = os.getcwd()
filepath = os.path.join(curr_dir, 'test.txt')

### os.path  
путь до файла

In [ ]:
# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath(filepath))


# возвращает имя файла / папки по полному пути до него
print(os.path.basename(filepath))


# проверить существование директории - True / False
print(os.path.exists(curr_dir))

/content/test.txt
test.txt
True


### os.listdir  
возвращает список файлов в данной директории

In [ ]:
os.listdir(curr_dir)

['.config', 'sample_data']

При обходе файлов не забывайте исключать системные директории, такие как .DS_Store

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
for root, dirs, files in os.walk(curr_dir):
    for name in files:
        print(os.path.join(root, name))

/content/.config/.last_update_check.json
/content/.config/.last_opt_in_prompt.yaml
/content/.config/.last_survey_prompt.yaml
/content/.config/config_sentinel
/content/.config/gce
/content/.config/active_config
/content/.config/.metricsUUID
/content/.config/logs/2020.08.27/16.38.54.457558.log
/content/.config/logs/2020.08.27/16.38.36.398860.log
/content/.config/logs/2020.08.27/16.39.27.968323.log
/content/.config/logs/2020.08.27/16.39.28.581989.log
/content/.config/logs/2020.08.27/16.39.08.060972.log
/content/.config/logs/2020.08.27/16.39.13.109523.log
/content/.config/configurations/config_default
/content/sample_data/anscombe.json
/content/sample_data/README.md
/content/sample_data/mnist_train_small.csv
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/california_housing_train.csv


> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(curr_dir))

[('/content', ['.config', 'sample_data'], []),
 ('/content/.config',
  ['logs', 'configurations'],
  ['.last_update_check.json',
   '.last_opt_in_prompt.yaml',
   '.last_survey_prompt.yaml',
   'config_sentinel',
   'gce',
   'active_config',
   '.metricsUUID']),
 ('/content/.config/logs', ['2020.08.27'], []),
 ('/content/.config/logs/2020.08.27',
  [],
  ['16.38.54.457558.log',
   '16.38.36.398860.log',
   '16.39.27.968323.log',
   '16.39.28.581989.log',
   '16.39.08.060972.log',
   '16.39.13.109523.log']),
 ('/content/.config/configurations', [], ['config_default']),
 ('/content/sample_data',
  [],
  ['anscombe.json',
   'README.md',
   'mnist_train_small.csv',
   'mnist_test.csv',
   'california_housing_test.csv',
   'california_housing_train.csv'])]

### чтение файла 

In [ ]:
fpath = 'test.txt'


# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

    
#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

    
#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 

FileNotFoundError: ignored

Напоминание про enumerate:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, не надо так

### CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
 
# инициализируем
vectorizer = CountVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        | слово1 | слово2 | слово3 | слово4
# текст1 |   1    |    1   |   1    |   0
# текст2 |   0    |    1   |   1    |   0
# текст3 |   2    |    1   |   0    |   0
# текст4 |   0    |    0   |   0    |   1
 
# чтобы получить сгенерированный словарь, из приведенной структуры CountVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('слово3') # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
vectorizer.transform(["слово1 слово4 слово4"])  # результат [[1 0 0 2]]
 
# чтобы узнать количественное вхождение каждого слова:
matrix_freq = np.asarray(X.sum(axis=0)).ravel()
final_matrix = np.array([np.array(vectorizer.get_feature_names()), matrix_freq])

In [ ]:
X.toarray()

array([[1, 1, 1, 0],
       [0, 1, 1, 0],
       [2, 1, 0, 0],
       [0, 0, 0, 1]])

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзей. Одна серия - один документ.

**To do:** 

**1 Создайте индекс этой базы в формате json и в формате матрицы Term-Document**

Компоненты вашей реализации:
    - функция препроцессинга данных
    - функция индексирования данных

**2 С помощью обратного индекса в формате Term-Document посчитайте:** 


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) кто из главных героев статистически самый популярный? 



[download_friends_corpus](https://yadi.sk/d/4wmU7R8JL-k_RA?w=1)

Напоминание про defaultdict: 
> Если ваш код предполагает использование multiple values словаря, рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

> ```from collections import defaultdict
d = defaultdict(list)
d['example'].append('example1')
d['example'].append('example2')```

In [ ]:
### _check : в коллекции должно быть около 165 файлов

In [ ]:
! unzip friends-data.zip

In [ ]:
! pip install pymorphy2

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 4.1MB/s 


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def preprocess(f):
    sym = "0123456789.,?!…:;()[]-_|/\"'«»*{}<>@#$%^&№"
    s = []
    for line in f:
        words = line.strip().lower().split()
        for word in words:
            word = word.strip(sym)
            if word != '':
                word = morph.parse(word)[0].normal_form
                s.append(word)
    return s

In [ ]:
from tqdm import tqdm

def index_json(files):
    d_json = defaultdict(list)
    for filename in tqdm(files):
        name = filename.split(os.sep)[-1]
        with open(filename, 'r', encoding='utf-8') as f:
            text = preprocess(f)
            c = Counter(text)
            for w in c.keys():
                d_json[w].append((name, c[w]))
    return d_json

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd

def index_matrix(files):

    corpus = []
    
    for filename in tqdm(files):
        with open(filename, 'r', encoding='utf-8') as f:
            text = preprocess(f)
            corpus.append(' '.join(text))

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)

    return pd.DataFrame(X.toarray(), index=files, columns=vectorizer.get_feature_names())

In [ ]:
os.sep

'/'

In [ ]:
import os
curr_dir = os.getcwd()
dir = os.path.join(curr_dir, 'friends-data')

files = []
for r, d, f in os.walk(dir):
    if len(f) > 0:
        for file in f:
            files.append(os.path.join(r, file))

In [ ]:
j = index_json(files)

100%|██████████| 165/165 [00:58<00:00,  2.80it/s]


In [ ]:
m = index_matrix(files)

100%|██████████| 165/165 [01:00<00:00,  2.71it/s]


In [ ]:
characters = {}
for name in ['рэйчел', 'моника', 'фиби', 'джоуя', 'чендлер', 'росс']:
    characters[name] = m.sum()[name]
sorted(characters, key=characters.get, reverse=True)[0]

'росс'

In [ ]:
m.sum()['росс']

1016

In [ ]:
for name in ['рэйчел', 'моника', 'фиби', 'джоуя', 'чендлер', 'росс']:
    print(name, m.sum()[name])

рэйчел 237
моника 684
фиби 575
джоуя 682
чендлер 48
росс 1016


In [ ]:
m['ты']

/content/friends-data/Friends - season 3/Friends - 3x12 - The One With All The Jealousy.ru.txt                     73
/content/friends-data/Friends - season 3/Friends - 3x14 - The One With Phoebe's Ex-Partner.ru.txt                  87
/content/friends-data/Friends - season 3/Friends - 3x07 - The One With The Race Car Bed.ru.txt                     54
/content/friends-data/Friends - season 3/Friends - 3x19 - The One With The Tiny T-Shirt.ru.txt                     89
/content/friends-data/Friends - season 3/Friends - 3x15 - The One Where Ross And Rachel Take A Break (1).ru.txt    72
                                                                                                                   ..
/content/friends-data/Friends - season 6/Friends - 6x13 - The One With Rachel's Sister (1).ru.txt                  85
/content/friends-data/Friends - season 6/Friends - 6x11 - The One With The Apothecary Table.ru.txt                 66
/content/friends-data/Friends - season 6/Friends - 6x24 

In [ ]:
m.sum().sort_values().tail(1)[0]

11191

In [ ]:
m

,50,5000,after,again,ahh,all,and,are,au,aнгел,ban,bay,behind,bodington,bonjour,book,by,careless,cat,cd,cheese,china,chuck,city,co,com,cамый,cочельник,cтарый,cтоп,dais,dance,daysailer,delivery,dickin,dicky,didn,dings,discovery,done,...,яйцо,якобы,якорь,ям,ямагучить,ямочка,ямс,ян,январь,янки,янкиз,янковский,японец,япония,японский,яркий,ярко,ярковата,ярлык,ярлычок,ярмарка,ярость,ясмина,ясно,ясность,ясный,яхта,ящерица,ящик,ящичек,ёвить,ёй,ёкнуть,ёлка,ёлочный,ёпэрэсотэ,ёрл,ёрш,ёршик,ёще
/content/friends-data/Friends - season 3/Friends - 3x12 - The One With All The Jealousy.ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
/content/friends-data/Friends - season 3/Friends - 3x14 - The One With Phoebe's Ex-Partner.ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
/content/friends-data/Friends - season 3/Friends - 3x07 - The One With The Race Car Bed.ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0
/content/friends-data/Friends - season 3/Friends - 3x19 - The One With The Tiny T-Shirt.ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
/content/friends-data/Friends - season 3/Friends - 3x15 - The One Where Ross And Rachel Take A Break (1).ru.txt,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/content/friends-data/Friends - season 6/Friends - 6x13 - The One With Rachel's Sister (1).ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
/content/friends-data/Friends - season 6/Friends - 6x11 - The One With The Apothecary Table.ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
/content/friends-data/Friends - season 6/Friends - 6x24 - The One With The Proposal (1).ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0
/content/friends-data/Friends - season 6/Friends - 6x14 - The One Where Chandler Can't Cry (2).ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
list(m.columns[(np.count_nonzero(m, axis=0) == 165).nonzero()])

['быть',
 'весь',
 'да',
 'думать',
 'если',
 'ещё',
 'знать',
 'как',
 'мой',
 'мочь',
 'мы',
 'на',
 'не',
 'нет',
 'но',
 'ну',
 'он',
 'она',
 'просто',
 'сказать',
 'так',
 'такой',
 'то',
 'тот',
 'ты',
 'хотеть',
 'что',
 'это',
 'этот']